In [4]:
# ! pip install pyrtools
# ! pip install opencv-python

In [236]:
from scipy import signal

def conv2(x, y, mode='same'):
    return np.rot90(signal.convolve2d(np.rot90(x, 2), np.rot90(y, 2), mode=mode), 2)


def RRoverlapconv(kernel, in_):
    out = conv2(in_, kernel, mode='same')
    
    rect = np.ones_like(in_)

    overlapsum = conv2(rect, kernel, 'same')
    out = np.sum(kernel) * out / overlapsum
    return out



def RRgaussfilter1D(halfsupport, sigma, center=0):
    t = list(range(-halfsupport, halfsupport+1))
    kernel = [np.exp(-(x-center) **2 /(2* sigma ** 2)) for x in t]
    kernel = kernel/sum(kernel)
    
    return kernel
    

def computeColorClutter(L_pyr, a_pyr, b_pyr, sigD):
    
    numlevels = len(L_pyr);
    
    covMx = {}
    clutter_levels = np.zeros([numlevels, 1])
    DL = [0] * numlevels
    Da = [0] * numlevels
    Db = [0] * numlevels
    
    
    deltaL2 = 0.0007 ** 2
    deltaa2 = 0.1 ** 2
    deltab2 = 0.05 ** 2
    
    bigG = RRgaussfilter1D(round(2*sigD), sigD)[:,None]
    
    for i in range(0, numlevels):
        DL[i] = RRoverlapconv(bigG, L_pyr[(i,0)])
        DL[i] = RRoverlapconv(bigG.T, DL[i])   # E(L)
        Da[i] = RRoverlapconv(bigG, a_pyr[(i,0)])
        Da[i] = RRoverlapconv(bigG.T, Da[i])   # E(a)
        Db[i] = RRoverlapconv(bigG, b_pyr[(i,0)]);
        Db[i] = RRoverlapconv(bigG.T, Db[i])    # E(b)

    
        # dict idea
        covMx[(i,0,0)] = RRoverlapconv(bigG, L_pyr[(i,0)] ** 2)
        covMx[(i,0,0)] = RRoverlapconv(bigG.T, covMx[(i,0,0)]) - DL[i] ** 2 + deltaL2  # cov(L,L) + deltaL2
        covMx[(i,0,1)] = RRoverlapconv(bigG, L_pyr[(i,0)] * a_pyr[(i,0)])
        covMx[(i,0,1)] = RRoverlapconv(bigG.T, covMx[(i,0,1)]) - DL[i] * Da[i]        # cov(L,a)
        covMx[(i,0,2)] = RRoverlapconv(bigG, L_pyr[(i,0)] * b_pyr[(i,0)])
        covMx[(i,0,2)] = RRoverlapconv(bigG.T, covMx[(i,0,2)]) - DL[i] * Db[i]        # cov(L,b)
        covMx[(i,1,1)] = RRoverlapconv(bigG, a_pyr[(i,0)] ** 2)
        covMx[(i,1,1)] = RRoverlapconv(bigG.T, covMx[(i,1,1)]) - Da[i] ** 2 + deltaa2  # cov(a,a) + deltaa2
        covMx[(i,1,2)] = RRoverlapconv(bigG, a_pyr[(i,0)] * b_pyr[(i,0)])
        covMx[(i,1,2)] = RRoverlapconv(bigG.T, covMx[(i,1,2)]) - Da[i] * Db[i]        # cov(a,b)
        covMx[(i,2,2)] = RRoverlapconv(bigG, b_pyr[(i,0)] ** 2)    
        covMx[(i,2,2)] = RRoverlapconv(bigG.T, covMx[(i,2,2)]) - Db[i] ** 2 + deltab2;  # cov(b,b) + deltab2

    
    if len(a_pyr)!=numlevels or len(b_pyr)!=numlevels:
        print('L, a, and b channels must have the same number of levels in the Gaussian pyramid\n')


    clutter_levels = numlevels
    return covMx

In [237]:
# test

RRLab = computeClutter('test.jpg')

computeColorClutter(RRLab[0],RRLab[1],RRLab[2], 3)


{(0,
  0,
  0): array([[711.33773942, 590.49487618, 472.78952136, ...,   1.73163504,
           1.49743428,   1.34980318],
        [654.38549646, 542.40782422, 435.41111408, ...,   1.60732943,
           1.40696864,   1.27223579],
        [595.32324943, 493.29765574, 397.43695445, ...,   1.45753453,
           1.29229859,   1.17453128],
        ...,
        [ 58.37046258,  65.78249226,  79.07540356, ...,   2.74082454,
           2.89982782,   2.9946377 ],
        [ 47.30484813,  52.19564524,  61.96258849, ...,   2.75305916,
           2.94765319,   3.07815804],
        [ 38.11224561,  42.01703119,  49.7857475 , ...,   2.66977181,
           2.88944029,   3.04684479]]),
 (0,
  0,
  1): array([[43.29633651, 30.99126058, 16.90995222, ..., -0.46623431,
         -0.63846531, -0.75872187],
        [41.51570752, 28.11140055, 13.82162944, ..., -0.41401163,
         -0.5762304 , -0.69259487],
        [38.76590731, 24.63051164, 10.45483949, ..., -0.35727135,
         -0.50592408, -0.614169  ],
 

In [218]:
bigG = RRgaussfilter1D(round(2*sigD), sigD)[:,None]
RRoverlapconv(bigG, L_pyr[(i,0)]).shape

(450, 450)

In [231]:
RRoverlapconv(bigG, L_pyr[(i,0)] ** 2).shape

(450, 450)

In [222]:
np.sum(bigG.T)

1.0000000000000002

In [ ]:
import pyrtools as pt
import numpy as np
from scipy import signal

def colorClutter(inputImage, numlevels, pool_sigma=3, pix=1):
    
    if isinstance(inputImage, list):
        L_pyr = RRLab[0]
        a_pyr = RRLab[1]
        b_pyr = RRLab[2]
        
    else:
        pass #TODO
    
    clutter_levels = computeColorClutter(L_pyr, a_pyr, b_pyr, pool_sigma);
    

    kernel_1d = np.array([[0.05, 0.25, 0.4, 0.25, 0.05]])
    kernel_2d = signal.convolve2d(kernel_1d, kernel_1d.T)
    
    #TODO
    
    return clutter_levels, clutter_map

In [82]:
import cv2
import pyrtools as pt
import numpy as np

def computeClutter(inputImage, numlevels=3, contrast_filt_sigma=1, contrast_pool_sigma=None, color_pool_sigma=3, contrast_pix=0, color_pix=0, orient_pix=0):
    
    if contrast_pool_sigma is None:
        contrast_pool_sigma = 3 * contrast_filt_sigma

    orient_pool_sigma = 7/2

    if isinstance(inputImage, str):
        im = cv2.imread(inputImage)
        if im is None:
            print('Unable to open %s image file.') #TODO: add logger
            return 0
        else:
            m, n, d = im.shape
            if d == 3:
                Lab = cv2.cvtColor(im, cv2.COLOR_RGB2LAB)
                RRLab = [0, 0, 0]
            else:
                print('should be run on RGB color images.  Input image appears to be grayscale.\n')
                return 0
  
    else:
        pass #TODO 

    pyr = pt.pyramids.GaussianPyramid(Lab[:,:,0], height=numlevels)
    RRLab[0] = pyr.pyr_coeffs

    pyr = pt.pyramids.GaussianPyramid(Lab[:,:,1], height=numlevels)
    RRLab[1] = pyr.pyr_coeffs

    pyr = pt.pyramids.GaussianPyramid(Lab[:,:,0], height=numlevels)
    RRLab[2] = pyr.pyr_coeffs
        
    
#     # compute the color clutter
#     color_clutter_levels, color_clutter_map = colorClutter(RRLab, numlevels, color_pool_sigma, color_pix)
#     # compute the contrast clutter
#     contrast_clutter_levels, contrast_clutter_map = contrastClutter(RRLab, numlevels, contrast_filt_sigma, contrast_pool_sigma, contrast_pix)
#     #compute the orientation clutter
#     orient_clutter_levels, orientation_clutter_map = orientationClutter(RRLab, numlevels, orient_pix)

#     # output them in cell structures
#     color_clutter = [color_clutter_levels, color_clutter_map]
#     contrast_clutter = [contrast_clutter_levels, contrast_clutter_map]
#     orientation_clutter = [orient_clutter_levels, orientation_clutter_map]

    return RRLab
    # return color_clutter, contrast_clutter, orientation_clutter


In [ ]:
def getClutter_Fc(filename, p=1):
    color_clutter, contrast_clutter, orient_clutter = computeClutter(filename, 3, 1, 3, 3, 0, 0, 0)
    clutter_map_fc = color_clutter{2} / 0.2088 + contrast_clutter{2} / 0.0660 + orient_clutter{2} / 0.0269
    clutter_scalar_fc = mean(clutter_map_fc(:).^ p).^ (1 / p) #element wise
    return clutter_scalar_fc, clutter_map_fc


In [ ]:
clutter_scalar_fc, clutter_map_fc = getClutter_FC('test.jpg');